In [93]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [94]:
# Parametros de seguridad
umbral_pico = 0.5

max_i_r_ple1 = 17
max_i_s_ple1 = 20
max_i_t_ple1 = 17

max_i_r_ple7 = 13
max_i_s_ple7 = 30
max_i_t_ple7 = 35

tiempo_min = pd.Timedelta(minutes=5)
cant_min_picos = 10
tiempo_min_picos = pd.Timedelta(minutes=60)



In [96]:
# Diccionario de parametros de seguridad

limites_corriente = {
    'PLE1': {
        'R': max_i_r_ple1,
        'S': max_i_s_ple1,
        'T': max_i_t_ple1,
    },
    'PLE7': {
        'R': max_i_r_ple7,
        'S': max_i_s_ple7,
        'T': max_i_t_ple7,
    }
}

In [97]:
# Abrir los graficos en el navegador

pio.renderers.default = 'browser'

In [98]:
# Funcion para cargar datos

from glob import glob
import pandas as pd

def cargar_maquina(base_path, maquina):
    paths = glob(f"{base_path}/{maquina}/**/*.csv", recursive=True)

    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        df['maquina'] = maquina
        dfs.append(df)

    return (
        pd.concat(dfs, ignore_index=True)
          .sort_values('temporal_placa')
          .reset_index(drop=True)
    )


In [99]:
# Carga de datos

df_ple1 = cargar_maquina("../data", "PLE1")
df_ple7 = cargar_maquina("../data", "PLE7")

In [100]:
# Funcion preparar_df

def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
        
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [101]:
# Ajuste de datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

In [102]:
# Funcion detectar_picos

def detectar_picos(signal, timestamp, umbral_pico):
    dt = timestamp.diff().dt.total_seconds()
    valid = dt > 0

    dI_dt = signal.diff() / dt

    return (
        valid &
        dI_dt.notna() &
        (dI_dt > umbral_pico)
    )

In [103]:
# Calculo la media de las corrientes excluyendo los picos

def media_sin_picos(s, q = 0.9):
    return s[s <= s.quantile(q)].mean()

medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        i_r_media=('corriente_r', media_sin_picos),
        i_s_media=('corriente_s', media_sin_picos),
        i_t_media=('corriente_t', media_sin_picos),
    )
)

medias_por_fase

,i_r_media,i_s_media,i_t_media
maquina,,,
PLE1,16.187595,17.210818,16.276914
PLE7,11.065071,23.275452,26.435411


In [104]:
# Visualizacion de picos detectados y su marca temporal

df_peaks_all = df_peaks_all.sort_values(
    by=['maquina', 'temporal_placa']
).reset_index(drop=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

#df_peaks_all

In [ ]:
# Analisis de Eventos
import pandas as pd

SILENCIO_MAX = pd.Timedelta(minutes=0.7)

rows = []

for maq, g in df_peaks_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').reset_index(drop=True)

    # diferencias con vecino anterior y siguiente
    g['dt_prev'] = g['temporal_placa'] - g['temporal_placa'].shift(1)
    g['dt_next'] = g['temporal_placa'].shift(-1) - g['temporal_placa']

    # pico espurio: aislado por ambos lados
    g['pico_espurio'] = (
        (g['dt_prev'] > 3 * SILENCIO_MAX) &
        (g['dt_next'] > 3 * SILENCIO_MAX)
    )

    # filtrar picos espurios
    g_filtrado = g[~g['pico_espurio']].copy()

    # diferencia entre picos consecutivos (CORRECTO)
    g_filtrado['dt'] = g_filtrado['temporal_placa'].diff()

    # nuevo paso
    g_filtrado['nuevo_paso'] = (
        g_filtrado['dt'].isna() |
        (g_filtrado['dt'] > SILENCIO_MAX)
    )

    g_filtrado['paso_id'] = g_filtrado['nuevo_paso'].cumsum()

    # agrupar por paso
    for paso_id, c in g_filtrado.groupby('paso_id'):

        fecha_inicio = c['temporal_placa'].iloc[0]
        fecha_fin    = c['temporal_placa'].iloc[-1]

        rows.append({
            'maquina': maq,
            'paso_id': paso_id,
            'fecha_inicio': fecha_inicio,
            'fecha_fin': fecha_fin,
            'duracion': fecha_fin - fecha_inicio
        })

df_pasos = pd.DataFrame(rows)

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None) 

df_pasos


,maquina,paso_id,fecha_inicio,fecha_fin,duracion
0,PLE7,1,2025-12-30 00:20:33-03:00,2025-12-30 00:20:33-03:00,0 days 00:00:00
1,PLE7,2,2025-12-30 07:26:31-03:00,2025-12-30 07:26:31-03:00,0 days 00:00:00
2,PLE7,3,2025-12-30 07:43:31-03:00,2025-12-30 07:43:31-03:00,0 days 00:00:00
3,PLE7,4,2025-12-30 08:19:33-03:00,2025-12-30 08:19:33-03:00,0 days 00:00:00
4,PLE7,5,2025-12-30 08:58:01-03:00,2025-12-30 08:58:01-03:00,0 days 00:00:00
5,PLE7,6,2025-12-30 11:23:31-03:00,2025-12-30 11:23:31-03:00,0 days 00:00:00
6,PLE7,7,2025-12-30 11:26:31-03:00,2025-12-30 11:26:31-03:00,0 days 00:00:00
7,PLE7,8,2025-12-30 11:33:02-03:00,2025-12-30 11:33:02-03:00,0 days 00:00:00
8,PLE7,9,2025-12-30 23:14:31-03:00,2025-12-30 23:14:31-03:00,0 days 00:00:00
9,PLE7,10,2026-01-09 17:26:57-03:00,2026-01-09 17:26:57-03:00,0 days 00:00:00


In [ ]:
# Filtrado de picos + timestamp para alarmas

peaks_rows = []

for maq, g in df_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').reset_index(drop=True)

    for fase, col in {'r': 'corriente_r',
                      's': 'corriente_s',
                      't': 'corriente_t'}.items():

        media_ref = medias_por_fase.loc[maq, f'i_{fase}_media']

        # 1) detectar flancos por derivada
        mask_picos = detectar_picos(
            g[col],
            g['temporal_placa'],
            umbral_pico
        )

        # 2) filtrar por amplitud: 3 × media base
        mask_grande = g[col] >= 2.4 * media_ref

        # 3) pico válido = derivada fuerte + amplitud grande
        mask_final = mask_picos & mask_grande

        p = g[mask_final]

        for _, r in p.iterrows():
            peaks_rows.append({
                'maquina': maq,
                'fase': fase,
                'temporal_placa': r['temporal_placa'],
                'valor_corriente': r[col],
                'media_base': media_ref
            })

df_peaks_alarmas = pd.DataFrame(peaks_rows)


In [ ]:
# Alarmas

alarmas = []

# A) Sobrecorriente sostenida por fase

for maq, g in df_all.groupby('maquina'):

    g = g.sort_values('temporal_placa')

    if maq not in limites_corriente:
        continue

    for fase, col in {'R':'corriente_r','S':'corriente_s','T':'corriente_t'}.items():

        limite = limites_corriente[maq][fase]

        g_fase = g[['temporal_placa', col]].copy()
        g_fase['exceso'] = g_fase[col] > limite
        g_fase['bloque'] = (g_fase['exceso'] != g_fase['exceso'].shift()).cumsum()

        for _, b in g_fase[g_fase['exceso']].groupby('bloque'):

            duracion = b['temporal_placa'].iloc[-1] - b['temporal_placa'].iloc[0]

            if duracion >= tiempo_min:
                alarmas.append({
                    'maquina': maq,
                    'tipo_alarma': 'SOBRECORRIENTE',
                    'descripcion': 'Sobrecorriente sostenida por encima del límite',
                    'fase': fase,
                    'valor_medido': b[col].mean(),
                    'valor_limite': limite,
                    'fecha_primera_deteccion': b['temporal_placa'].iloc[0],
                    'duracion_min': duracion.total_seconds() / 60
                })


# B) Frecuencia anómala de picos severos

df_peaks_all = (
    df_peaks_all
    .sort_values('temporal_placa')
    .drop_duplicates(subset=['maquina','fase','temporal_placa'])
)

for (maq, fase), gp in df_peaks_alarmas.groupby(['maquina','fase']):

    gp = gp.sort_values('temporal_placa').set_index('temporal_placa')

    s = pd.Series(1, index=gp.index)
    cnt = s.rolling('60min').sum()

    viol = cnt[cnt > cant_min_picos]

    if not viol.empty:
        t_fin = viol.index[0]
        t_ini = t_fin - tiempo_min_picos

        alarmas.append({
            'maquina': maq,
            'tipo_alarma': 'FRECUENCIA_PICOS',
            'descripcion': 'Frecuencia anómala de picos severos de corriente (>10 en 60 min)',
            'fase': fase.upper(),
            'valor_medido': cnt.max()
,
            'valor_limite': cant_min_picos,
            'fecha_primera_deteccion': t_ini,
            'duracion_min': tiempo_min_picos.total_seconds() / 60
        })

df_alarmas = pd.DataFrame(alarmas)
df_alarmas


,maquina,tipo_alarma,descripcion,fase,valor_medido,valor_limite,fecha_primera_deteccion,duracion_min
0,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,R,17.316908,17,2026-01-10 00:13:59-03:00,11.483333
1,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,18.771630,17,2025-12-30 00:00:14-03:00,53.300000
2,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.262980,17,2025-12-30 05:08:32-03:00,5.500000
3,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.146687,17,2025-12-30 05:53:03-03:00,5.983333
4,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.354562,17,2025-12-30 11:59:31-03:00,22.016667
5,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.235594,17,2025-12-30 22:14:32-03:00,10.483333
6,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.730256,17,2026-01-09 18:28:43-03:00,10.100000
7,PLE1,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,T,17.885173,17,2026-01-09 18:45:16-03:00,6.850000
8,PLE7,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,R,13.211441,13,2025-12-30 00:00:01-03:00,8.033333
9,PLE7,SOBRECORRIENTE,Sobrecorriente sostenida por encima del límite,R,13.997849,13,2025-12-30 00:09:01-03:00,17.516667
